# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

In [1]:
#student names:
# Nikita Borochov
# Matanel Shachamorov

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

In [4]:
from sklearn.metrics import make_scorer

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [5]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [6]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [7]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [8]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [9]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [10]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [11]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

# Functions:

In [12]:
# YOUR CODE HERE
def clean_words(df):
    
    #clean non-hebrew characters
    df['story'] = df['story'].str.replace(r'[^\u0590-\u05fe\s]+', '', regex=True)
    
    for i in range(df.shape[0]):

        #clean prefix
        df['story'][i] = re.sub(r'\b(וכש|לכש)', '', df['story'][i])
        df['story'][i] = re.sub(r'\b(שה|וש|כש|)', '', df['story'][i])
        df['story'][i] = re.sub(r'\b(כ|ו|ש|ב|ה|ל)', '', df['story'][i])
        
        #clean words not ending with "female letters"
        df['story'][i] =' '.join(re.findall(r'\b\w+(?:ת|ה|י|ו)\b',df['story'][i]))
        
        #clean short words
        df['story'][i] = re.sub(r'\b\w{1,2}\b', '', df['story'][i])

    return df

In [13]:
def CVDFizer(df):
    #creates count vector
    vec = CountVectorizer(ngram_range=(1,3),min_df=5)
    x_train = vec.fit_transform(df["story"])
    df_vec = pd.DataFrame(x_train.toarray(), columns=vec.get_feature_names_out())
    df_vec["gender"] = df["gender"]
    
    return df_vec

In [14]:
def TFIDFizer(df):
    #creates TFIDF vector
    vec = TfidfVectorizer(ngram_range=(1,3),min_df=5)
    x_train_idf = vec.fit_transform(df["story"])
    df_vec = pd.DataFrame(x_train_idf.toarray(), columns=vec.get_feature_names_out())
    df_vec["gender"] = df["gender"]
    
    return df_vec

In [15]:
def womanizer(df):
    #creates 2 dataframes:
    #female where f=1 and m=0
    #male where f=0 and m=1
    
    female = df.replace("f", 1)
    female = female.replace("m", 0)
    male = df.replace("m", 1)
    male = male.replace("f", 0)
    
    return female, male

In [16]:
def cross_score_avg(clf, df_male, df_female):
    #function returns the average f1 score of female and male based classification models 
    score_female = cross_val_score(clf, df_female.drop(columns = ["gender"]), df_female["gender"],
                                   scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
    
    score_male = cross_val_score(clf, df_male.drop(columns = ["gender"]), df_male["gender"],
                                 scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
    
    return ((score_female.mean()+score_male.mean())/2)

# Research Train Data:

### cleaning the data:

In [17]:
df_clean = clean_words(df_train)
df_clean

,story,gender
0,אותי אמת חשבתי יקרה אמרתי תיארתי עצני אחרי א...,m
1,פני תגייסתי עשיתי מני יחידות מובחרות סיירות או...,m
2,תחילו חלומו פנגויה קבוצות מכללה אחרי צלחתי ע...,f
3,ייתי אותי טייסת קריאו פני שינה חמשת מלחמות ...,m
4,ייתי מדריכה יינו אני ייתי מדריכה תלמידי יתות א...,f
...,...,...
748,פני טסתי יינו החלטנו אנחנו רצינות דיברנ...,m
749,עליתי ספונטני משפחתי תלבטויות חלטנו אנחנו אילת...,m
750,פני עברנו דירה בית עקבות משפחתי מתגוררת דירה י...,m
751,חוויה אותה ארצה תרחשה פני קיצי קבעתי שבת ...,f


### vectorizing:

In [18]:

dfcv = CVDFizer(df_clean)
dfidf = TFIDFizer(df_clean)

In [47]:
dfcv.head()
dfidf.head()

אבטחה  אבי  אדומות  אדמה  אהבה  אהבתי  אהובה  אהובות  אהיה  אודות  אוהבת  \
0      0    0       0     0     0      0      0       0     0      0      0   
1      0    0       0     0     0      0      0       0     0      0      0   
2      0    0       0     0     0      0      0       0     0      0      0   
3      0    0       0     0     0      0      0       0     0      0      0   
4      0    0       0     0     0      0      0       0     0      0      0   

   אווירה  אווירה טובה  אווירה ייתה  אווירת  אוזניות  אוטו  אוטוטו  אוטומטי  \
0       0            0            0       0        0     2       0        0   
1       0            0            0       0        0     0       0        0   
2       0            0            0       0        0     0       0        0   
3       0            0            0       0        0     0       0        0   
4       0            0            0       0        0     0       0        1   

   אוי  אוירה  אוכלוסייה  אולי  אולי אני  אולי אפילו  אולי יהיה  אולי קצת  \
0    0      0          0     0         0           0          0         0   
1    0      0          0     0         0           0          0         0   
2    0      0          0     0         0           0          0         0   
3    0      0          0     0         0           0          0         0   
4    0      0          0     0         0           0          0         0   

   אולמות  אומנה  אומנות  אומרת  אופטימיות  אופי  אופציה  אופציות  אוקיי  \
0       0      0       0      0          0     0       0        0      1   
1       0      0       0      0          0     0       0        0      0   
2       0      0       0      0          0     0       0        0      0   
3       0      0       0      0          0     0       0        0      0   
4       0      0       0      0          0     0       0        0      0   

   אורות  אורכו  אות  אותה  אותה אחרי  אותה אני  אותה חברה  אותה קצת  \
0      0      0    0     0          0         0          0         0   
1      0      0    0     0          0         0          0         0   
2      0      0    0     1          0         0          0         0   
3      0      0    0     0          0         0          0         0   
4      0      0    0     0          0         0          0         0   

   אותה תקופה  אותו  אותו אותה  אותו אותו  אותו אותי  אותו אחרי  אותו איתנו  \
0           0     1          0          0          0          0           0   
1           0     1          0          0          0          0           0   
2           0     1          0          0          0          0           0   
3           0     3          0          0          0          0           0   
4           0     1          0          0          0          0           0   

   אותו אנחנו  אותו אני  אותו בית  אותו ביתה  אותו היות  אותו זאת  אותו חזרה  \
0           0         0         0          0          0         1          0   
1           0         0         0          0          0         0          0   
2           0         0         0          0          0         0          0   
3           0         0         0          0          0         0          0   
4           0         0         0          0          0         0          0   

   אותו יהיה  אותו יינו  אותו ייתי  אותו מנת  אותו מקרה  אותו פני  \
0          0          0          0         0          0         0   
1          0          0          0         0          0         0   
2          0          0          0         0          0         0   
3          0          0          0         0          1         0   
4          0          0          0         0          0         0   

   אותו קיבלתי  אותו קרה  אותו רבה  אותו תחלתי  אותי  אותי אותו  אותי אותי  \
0            0         0         0           0     1          0          0   
1            1         0         0           0     0          0          0   
2            0         0         0           0     0          0          0   
3            0   

אבטחה   אבי  אדומות  אדמה  אהבה  אהבתי  אהובה  אהובות  אהיה  אודות  אוהבת  \
0  0.000 0.000   0.000 0.000 0.000  0.000  0.000   0.000 0.000  0.000  0.000   
1  0.000 0.000   0.000 0.000 0.000  0.000  0.000   0.000 0.000  0.000  0.000   
2  0.000 0.000   0.000 0.000 0.000  0.000  0.000   0.000 0.000  0.000  0.000   
3  0.000 0.000   0.000 0.000 0.000  0.000  0.000   0.000 0.000  0.000  0.000   
4  0.000 0.000   0.000 0.000 0.000  0.000  0.000   0.000 0.000  0.000  0.000   

   אווירה  אווירה טובה  אווירה ייתה  אווירת  אוזניות  אוטו  אוטוטו  אוטומטי  \
0   0.000        0.000        0.000   0.000    0.000 0.130   0.000    0.000   
1   0.000        0.000        0.000   0.000    0.000 0.000   0.000    0.000   
2   0.000        0.000        0.000   0.000    0.000 0.000   0.000    0.000   
3   0.000        0.000        0.000   0.000    0.000 0.000   0.000    0.000   
4   0.000        0.000        0.000   0.000    0.000 0.000   0.000    0.116   

    אוי  אוירה  אוכלוסייה  אולי  אולי אני  אולי אפילו  אולי יהיה  אולי קצת  \
0 0.000  0.000      0.000 0.000     0.000       0.000      0.000     0.000   
1 0.000  0.000      0.000 0.000     0.000       0.000      0.000     0.000   
2 0.000  0.000      0.000 0.000     0.000       0.000      0.000     0.000   
3 0.000  0.000      0.000 0.000     0.000       0.000      0.000     0.000   
4 0.000  0.000      0.000 0.000     0.000       0.000      0.000     0.000   

   אולמות  אומנה  אומנות  אומרת  אופטימיות  אופי  אופציה  אופציות  אוקיי  \
0   0.000  0.000   0.000  0.000      0.000 0.000   0.000    0.000  0.108   
1   0.000  0.000   0.000  0.000      0.000 0.000   0.000    0.000  0.000   
2   0.000  0.000   0.000  0.000      0.000 0.000   0.000    0.000  0.000   
3   0.000  0.000   0.000  0.000      0.000 0.000   0.000    0.000  0.000   
4   0.000  0.000   0.000  0.000      0.000 0.000   0.000    0.000  0.000   

   אורות  אורכו   אות  אותה  אותה אחרי  אותה אני  אותה חברה  אותה קצת  \
0  0.000  0.000 0.000 0.000      0.000     0.000      0.000     0.000   
1  0.000  0.000 0.000 0.000      0.000     0.000      0.000     0.000   
2  0.000  0.000 0.000 0.046      0.000     0.000      0.000     0.000   
3  0.000  0.000 0.000 0.000      0.000     0.000      0.000     0.000   
4  0.000  0.000 0.000 0.000      0.000     0.000      0.000     0.000   

   אותה תקופה  אותו  אותו אותה  אותו אותו  אותו אותי  אותו אחרי  אותו איתנו  \
0       0.000 0.029      0.000      0.000      0.000      0.000       0.000   
1       0.000 0.044      0.000      0.000      0.000      0.000       0.000   
2       0.000 0.035      0.000      0.000      0.000      0.000       0.000   
3       0.000 0.106      0.000      0.000      0.000      0.000       0.000   
4       0.000 0.032      0.000      0.000      0.000      0.000       0.000   

   אותו אנחנו  אותו אני  אותו בית  אותו ביתה  אותו היות  אותו זאת  אותו חזרה  \
0       0.000     0.000     0.000      0.000      0.000     0.102      0.000   
1       0.000     0.000     0.000      0.000      0.000     0.000      0.000   
2       0.000     0.000     0.000      0.000      0.000     0.000      0.000   
3       0.000     0.000     0.000      0.000      0.000     0.000      0.000   
4       0.000     0.000     0.000      0.000      0.000     0.000      0.000   

   אותו יהיה  אותו יינו  אותו ייתי  אותו מנת  אותו מקרה  אותו פני  \
0      0.000      0.000      0.000     0.000      0.000     0.000   
1      0.000      0.000      0.000     0.000      0.000     0.000   
2      0.000      0.000      0.000     0.000      0.000     0.000   
3      0.000      0.000      0.000     0.000      0.126     0.000   
4      0.000      0.000      0.000     0.000      0.000     0.000   

   אותו קיבלתי  אותו קרה  אותו רבה  אותו תחלתי  אותי  אותי אותו  אותי אותי  \
0        0.000     0.000     0.000       0.000 0.033      0.000      0.000   
1        0.164     0.000     0.000       0.000 0.000      0.000      0.000   
2        0.000     0.000     0.000       0.000 0.000      0.000      0.000   

### creating female and male dataframe for each vector:

In [48]:
df_female_cvec, df_male_cvec = womanizer(dfcv)
df_female_idf, df_male_idf = womanizer(dfidf)

In [50]:
print("male:0 , female:1")
df_female_cvec.head(5)

print("male:1 , female:0")
df_male_cvec.head(5)

male:0 , female:1


אבטחה  אבי  אדומות  אדמה  אהבה  אהבתי  אהובה  אהובות  אהיה  אודות  אוהבת  \
0      0    0       0     0     0      0      0       0     0      0      0   
1      0    0       0     0     0      0      0       0     0      0      0   
2      0    0       0     0     0      0      0       0     0      0      0   
3      0    0       0     0     0      0      0       0     0      0      0   
4      0    0       0     0     0      0      0       0     0      0      0   

   אווירה  אווירה טובה  אווירה ייתה  אווירת  אוזניות  אוטו  אוטוטו  אוטומטי  \
0       0            0            0       0        0     2       0        0   
1       0            0            0       0        0     0       0        0   
2       0            0            0       0        0     0       0        0   
3       0            0            0       0        0     0       0        0   
4       0            0            0       0        0     0       0        1   

   אוי  אוירה  אוכלוסייה  אולי  אולי אני  אולי אפילו  אולי יהיה  אולי קצת  \
0    0      0          0     0         0           0          0         0   
1    0      0          0     0         0           0          0         0   
2    0      0          0     0         0           0          0         0   
3    0      0          0     0         0           0          0         0   
4    0      0          0     0         0           0          0         0   

   אולמות  אומנה  אומנות  אומרת  אופטימיות  אופי  אופציה  אופציות  אוקיי  \
0       0      0       0      0          0     0       0        0      1   
1       0      0       0      0          0     0       0        0      0   
2       0      0       0      0          0     0       0        0      0   
3       0      0       0      0          0     0       0        0      0   
4       0      0       0      0          0     0       0        0      0   

   אורות  אורכו  אות  אותה  אותה אחרי  אותה אני  אותה חברה  אותה קצת  \
0      0      0    0     0          0         0          0         0   
1      0      0    0     0          0         0          0         0   
2      0      0    0     1          0         0          0         0   
3      0      0    0     0          0         0          0         0   
4      0      0    0     0          0         0          0         0   

   אותה תקופה  אותו  אותו אותה  אותו אותו  אותו אותי  אותו אחרי  אותו איתנו  \
0           0     1          0          0          0          0           0   
1           0     1          0          0          0          0           0   
2           0     1          0          0          0          0           0   
3           0     3          0          0          0          0           0   
4           0     1          0          0          0          0           0   

   אותו אנחנו  אותו אני  אותו בית  אותו ביתה  אותו היות  אותו זאת  אותו חזרה  \
0           0         0         0          0          0         1          0   
1           0         0         0          0          0         0          0   
2           0         0         0          0          0         0          0   
3           0         0         0          0          0         0          0   
4           0         0         0          0          0         0          0   

   אותו יהיה  אותו יינו  אותו ייתי  אותו מנת  אותו מקרה  אותו פני  \
0          0          0          0         0          0         0   
1          0          0          0         0          0         0   
2          0          0          0         0          0         0   
3          0          0          0         0          1         0   
4          0          0          0         0          0         0   

   אותו קיבלתי  אותו קרה  אותו רבה  אותו תחלתי  אותי  אותי אותו  אותי אותי  \
0            0         0         0           0     1          0          0   
1            1         0         0           0     0          0          0   
2            0         0         0           0     0          0          0   
3            0   

male:1 , female:0


אבטחה  אבי  אדומות  אדמה  אהבה  אהבתי  אהובה  אהובות  אהיה  אודות  אוהבת  \
0      0    0       0     0     0      0      0       0     0      0      0   
1      0    0       0     0     0      0      0       0     0      0      0   
2      0    0       0     0     0      0      0       0     0      0      0   
3      0    0       0     0     0      0      0       0     0      0      0   
4      0    0       0     0     0      0      0       0     0      0      0   

   אווירה  אווירה טובה  אווירה ייתה  אווירת  אוזניות  אוטו  אוטוטו  אוטומטי  \
0       0            0            0       0        0     2       0        0   
1       0            0            0       0        0     0       0        0   
2       0            0            0       0        0     0       0        0   
3       0            0            0       0        0     0       0        0   
4       0            0            0       0        0     0       0        1   

   אוי  אוירה  אוכלוסייה  אולי  אולי אני  אולי אפילו  אולי יהיה  אולי קצת  \
0    0      0          0     0         0           0          0         0   
1    0      0          0     0         0           0          0         0   
2    0      0          0     0         0           0          0         0   
3    0      0          0     0         0           0          0         0   
4    0      0          0     0         0           0          0         0   

   אולמות  אומנה  אומנות  אומרת  אופטימיות  אופי  אופציה  אופציות  אוקיי  \
0       0      0       0      0          0     0       0        0      1   
1       0      0       0      0          0     0       0        0      0   
2       0      0       0      0          0     0       0        0      0   
3       0      0       0      0          0     0       0        0      0   
4       0      0       0      0          0     0       0        0      0   

   אורות  אורכו  אות  אותה  אותה אחרי  אותה אני  אותה חברה  אותה קצת  \
0      0      0    0     0          0         0          0         0   
1      0      0    0     0          0         0          0         0   
2      0      0    0     1          0         0          0         0   
3      0      0    0     0          0         0          0         0   
4      0      0    0     0          0         0          0         0   

   אותה תקופה  אותו  אותו אותה  אותו אותו  אותו אותי  אותו אחרי  אותו איתנו  \
0           0     1          0          0          0          0           0   
1           0     1          0          0          0          0           0   
2           0     1          0          0          0          0           0   
3           0     3          0          0          0          0           0   
4           0     1          0          0          0          0           0   

   אותו אנחנו  אותו אני  אותו בית  אותו ביתה  אותו היות  אותו זאת  אותו חזרה  \
0           0         0         0          0          0         1          0   
1           0         0         0          0          0         0          0   
2           0         0         0          0          0         0          0   
3           0         0         0          0          0         0          0   
4           0         0         0          0          0         0          0   

   אותו יהיה  אותו יינו  אותו ייתי  אותו מנת  אותו מקרה  אותו פני  \
0          0          0          0         0          0         0   
1          0          0          0         0          0         0   
2          0          0          0         0          0         0   
3          0          0          0         0          1         0   
4          0          0          0         0          0         0   

   אותו קיבלתי  אותו קרה  אותו רבה  אותו תחלתי  אותי  אותי אותו  אותי אותי  \
0            0         0         0           0     1          0          0   
1            1         0         0           0     0          0          0   
2            0         0         0           0     0          0          0   
3            0   

### cross validation of different classifiers:

In [20]:

models = [SGDClassifier(), LinearSVC(), Perceptron(),DecisionTreeClassifier(),MultinomialNB(), GaussianNB()]
print("count vectorizer:")
for model in models:
    print(model)
    print(cross_score_avg(model, df_male_cvec, df_female_cvec))
    
print("\nTFIDF:")
for model in models:
    print(model)
    print(cross_score_avg(model, df_male_idf, df_female_idf))

count vectorizer:
SGDClassifier()
0.651276384087955
LinearSVC()
0.6598148613894955
Perceptron()
0.6766801765543835
DecisionTreeClassifier()
0.6471370243137924
MultinomialNB()
0.7380450949429368
GaussianNB()
0.5580518412067402

TFIDF:
SGDClassifier()
0.6715950871464427
LinearSVC()
0.6410820590360988
Perceptron()
0.6156131493143866
DecisionTreeClassifier()
0.6836706195796003
MultinomialNB()
0.4329778822905296
GaussianNB()
0.5473983344081952


### Grid search for best params:

#### count vectorizer:

In [21]:
# sgd
sgd_params = {
    "penalty": ["l1", "l2"],
    "loss": ["hinge", "log"],
    "max_iter": [10, 22, 50],
    "validation_fraction": [0.1, 0.2]
}
sgd = GridSearchCV(SGDClassifier(), sgd_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
sgd_female = sgd.fit(df_female_cvec.drop(columns = ["gender"]),df_female_cvec["gender"])
sgd_male = sgd.fit(df_male_cvec.drop(columns = ["gender"]),df_male_cvec["gender"])
sgd_female_model = sgd_female.best_estimator_
sgd_male_model = sgd_male.best_estimator_
sgd_female_score = sgd_female.best_score_
sgd_male_score = sgd_male.best_score_
sgd_female_params = sgd_female.best_params_
sgd_male_params = sgd_male.best_params_

sgd_female_params
sgd_female_model
sgd_female_score

sgd_male_params
sgd_male_model
sgd_male_score

(sgd_female_score+sgd_male_score)/2

{'loss': 'log', 'max_iter': 50, 'penalty': 'l2', 'validation_fraction': 0.1}

SGDClassifier(loss='log', max_iter=50)

0.6923500448951898

{'loss': 'log', 'max_iter': 50, 'penalty': 'l2', 'validation_fraction': 0.1}

SGDClassifier(loss='log', max_iter=50)

0.6923500448951898

0.6923500448951898

In [22]:
# linear svc
lin_svc_params = {
    "penalty": ["l1", "l2"],
    "loss": ["hinge", "squared_hinge"],
    "max_iter": [10, 22, 50],
    "class_weight" : ["balanced",None]
}
lin_svc = GridSearchCV(LinearSVC(), lin_svc_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
lin_svc_female = lin_svc.fit(df_female_cvec.drop(columns = ["gender"]),df_female_cvec["gender"])
lin_svc_male = lin_svc.fit(df_male_cvec.drop(columns = ["gender"]),df_male_cvec["gender"])
lin_svc_female_model = lin_svc_female.best_estimator_
lin_svc_male_model = lin_svc_male.best_estimator_
lin_svc_female_score = lin_svc_female.best_score_
lin_svc_male_score = lin_svc_male.best_score_
lin_svc_female_params = lin_svc_female.best_params_
lin_svc_male_params = lin_svc_male.best_params_

lin_svc_female_params
lin_svc_female_model
lin_svc_female_score

lin_svc_male_params
lin_svc_male_model
lin_svc_male_score

(lin_svc_female_score+lin_svc_male_score)/2

{'class_weight': 'balanced',
 'loss': 'squared_hinge',
 'max_iter': 50,
 'penalty': 'l2'}

LinearSVC(class_weight='balanced', max_iter=50)

0.6758437556524679

{'class_weight': 'balanced',
 'loss': 'squared_hinge',
 'max_iter': 50,
 'penalty': 'l2'}

LinearSVC(class_weight='balanced', max_iter=50)

0.6758437556524679

0.6758437556524679

In [23]:
# perceptron
per_params = {
    "penalty": ["l1", "l2","elasticnet"],
    "alpha":[0.0001,0.001],
    "max_iter": [10, 22, 50],
    "class_weight" : ["balanced",None]
}
per = GridSearchCV(Perceptron(), per_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
per_female = per.fit(df_female_cvec.drop(columns = ["gender"]),df_female_cvec["gender"])
per_male = per.fit(df_male_cvec.drop(columns = ["gender"]),df_male_cvec["gender"])
per_female_model = per_female.best_estimator_
per_male_model = per_male.best_estimator_
per_female_score = per_female.best_score_
per_male_score = per_male.best_score_
per_female_params = per_female.best_params_
per_male_params = per_male.best_params_

per_female_params
per_female_model
per_female_score

per_male_params
per_male_model
per_male_score

(per_female_score+per_male_score)/2

{'alpha': 0.0001, 'class_weight': None, 'max_iter': 22, 'penalty': 'l1'}

Perceptron(max_iter=22, penalty='l1')

0.680972932959161

{'alpha': 0.0001, 'class_weight': None, 'max_iter': 22, 'penalty': 'l1'}

Perceptron(max_iter=22, penalty='l1')

0.680972932959161

0.680972932959161

In [24]:
# Decision Tree
dtree_params = {
    "criterion": ["gini", "entropy","log_loss"],
    "splitter": ["best", "random"],
    "max_depth": [None,5,10,15],
    "min_samples_split": [2, 5,10]
}
dtree= GridSearchCV(DecisionTreeClassifier(), dtree_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
dtree_female = dtree.fit(df_female_cvec.drop(columns = ["gender"]),df_female_cvec["gender"])
dtree_male = dtree.fit(df_male_cvec.drop(columns = ["gender"]),df_male_cvec["gender"])
dtree_female_model = dtree_female.best_estimator_
dtree_male_model = dtree_male.best_estimator_
dtree_female_score = dtree_female.best_score_
dtree_male_score = dtree_male.best_score_
dtree_female_params = dtree_female.best_params_
dtree_male_params = dtree_male.best_params_

dtree_female_params
dtree_female_model
dtree_female_score

dtree_male_params
dtree_male_model
dtree_male_score

(dtree_female_score+dtree_male_score)/2

{'criterion': 'gini',
 'max_depth': 15,
 'min_samples_split': 2,
 'splitter': 'random'}

DecisionTreeClassifier(max_depth=15, splitter='random')

0.6929928635859228

{'criterion': 'gini',
 'max_depth': 15,
 'min_samples_split': 2,
 'splitter': 'random'}

DecisionTreeClassifier(max_depth=15, splitter='random')

0.6929928635859228

0.6929928635859228

In [25]:
# multinomialNB
mnb_params = {
    "alpha": [0,0.5,1.0,1.5,2]
}
mnb= GridSearchCV(MultinomialNB(), mnb_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
mnb_female = mnb.fit(df_female_cvec.drop(columns = ["gender"]),df_female_cvec["gender"])
mnb_male = mnb.fit(df_male_cvec.drop(columns = ["gender"]),df_male_cvec["gender"])
mnb_female_model = mnb_female.best_estimator_
mnb_male_model = mnb_male.best_estimator_
mnb_female_score = mnb_female.best_score_
mnb_male_score = mnb_male.best_score_
mnb_female_params = mnb_female.best_params_
mnb_male_params = mnb_male.best_params_

mnb_female_params
mnb_female_model
mnb_female_score

mnb_male_params
mnb_male_model
mnb_male_score

(mnb_female_score+mnb_male_score)/2

{'alpha': 0.5}

MultinomialNB(alpha=0.5)

0.740079761456341

{'alpha': 0.5}

MultinomialNB(alpha=0.5)

0.740079761456341

0.740079761456341

#### TFIDF

In [26]:
# sgd
sgd_params = {
    "penalty": ["l1", "l2"],
    "loss": ["hinge", "log"],
    "max_iter": [10, 22, 50],
    
    "validation_fraction": [0.1, 0.2]
}
sgd = GridSearchCV(SGDClassifier(), sgd_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
sgd_female = sgd.fit(df_female_idf.drop(columns = ["gender"]),df_female_idf["gender"])
sgd_male = sgd.fit(df_male_idf.drop(columns = ["gender"]),df_male_idf["gender"])
sgd_female_model = sgd_female.best_estimator_
sgd_male_model = sgd_male.best_estimator_
sgd_female_score = sgd_female.best_score_
sgd_male_score = sgd_male.best_score_
sgd_female_params = sgd_female.best_params_
sgd_male_params = sgd_male.best_params_

sgd_female_params
sgd_female_model
sgd_female_score

sgd_male_params
sgd_male_model
sgd_male_score

(sgd_female_score+sgd_male_score)/2

{'loss': 'hinge', 'max_iter': 22, 'penalty': 'l1', 'validation_fraction': 0.2}

SGDClassifier(max_iter=22, penalty='l1', validation_fraction=0.2)

0.6922881671014819

{'loss': 'hinge', 'max_iter': 22, 'penalty': 'l1', 'validation_fraction': 0.2}

SGDClassifier(max_iter=22, penalty='l1', validation_fraction=0.2)

0.6922881671014819

0.6922881671014819

In [27]:
# linear svc
lin_svc_params = {
    "penalty": ["l1", "l2"],
    "loss": ["hinge", "squared_hinge"],
    "max_iter": [10, 22, 50],
    "class_weight" : ["balanced",None]
}
lin_svc = GridSearchCV(LinearSVC(), lin_svc_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
lin_svc_female = lin_svc.fit(df_female_idf.drop(columns = ["gender"]),df_female_idf["gender"])
lin_svc_male = lin_svc.fit(df_male_idf.drop(columns = ["gender"]),df_male_idf["gender"])
lin_svc_female_model = lin_svc_female.best_estimator_
lin_svc_male_model = lin_svc_male.best_estimator_
lin_svc_female_score = lin_svc_female.best_score_
lin_svc_male_score = lin_svc_male.best_score_
lin_svc_female_params = lin_svc_female.best_params_
lin_svc_male_params = lin_svc_male.best_params_

lin_svc_female_params
lin_svc_female_model
lin_svc_female_score

lin_svc_male_params
lin_svc_male_model
lin_svc_male_score

(lin_svc_female_score+lin_svc_male_score)/2

{'class_weight': 'balanced', 'loss': 'hinge', 'max_iter': 22, 'penalty': 'l2'}

LinearSVC(class_weight='balanced', loss='hinge', max_iter=22)

0.6898250796865526

{'class_weight': 'balanced', 'loss': 'hinge', 'max_iter': 22, 'penalty': 'l2'}

LinearSVC(class_weight='balanced', loss='hinge', max_iter=22)

0.6898250796865526

0.6898250796865526

In [28]:
# perceptron
per_params = {
    "penalty": ["l1", "l2","elasticnet"],
    "alpha":[0.0001,0.001],
    "max_iter": [10, 22, 50],
    "class_weight" : ["balanced",None]
}
per = GridSearchCV(Perceptron(), per_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
per_female = per.fit(df_female_idf.drop(columns = ["gender"]),df_female_idf["gender"])
per_male = per.fit(df_male_idf.drop(columns = ["gender"]),df_male_idf["gender"])
per_female_model = per_female.best_estimator_
per_male_model = per_male.best_estimator_
per_female_score = per_female.best_score_
per_male_score = per_male.best_score_
per_female_params = per_female.best_params_
per_male_params = per_male.best_params_

per_female_params
per_female_model
per_female_score

per_male_params
per_male_model
per_male_score

(per_female_score+per_male_score)/2

{'alpha': 0.0001, 'class_weight': 'balanced', 'max_iter': 22, 'penalty': 'l1'}

Perceptron(class_weight='balanced', max_iter=22, penalty='l1')

0.6689720752113997

{'alpha': 0.0001, 'class_weight': 'balanced', 'max_iter': 22, 'penalty': 'l1'}

Perceptron(class_weight='balanced', max_iter=22, penalty='l1')

0.6689720752113997

0.6689720752113997

In [29]:
# Decision Tree
dtree_params = {
    "criterion": ["gini", "entropy","log_loss"],
    "splitter": ["best", "random"],
    "max_depth": [None,5,10,15],
    "min_samples_split": [2, 5,10]
}
dtree= GridSearchCV(DecisionTreeClassifier(), dtree_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
dtree_female = dtree.fit(df_female_idf.drop(columns = ["gender"]),df_female_idf["gender"])
dtree_male = dtree.fit(df_male_idf.drop(columns = ["gender"]),df_male_idf["gender"])
dtree_female_model = dtree_female.best_estimator_
dtree_male_model = dtree_male.best_estimator_
dtree_female_score = dtree_female.best_score_
dtree_male_score = dtree_male.best_score_
dtree_female_params = dtree_female.best_params_
dtree_male_params = dtree_male.best_params_

dtree_female_params
dtree_female_model
dtree_female_score

dtree_male_params
dtree_male_model
dtree_male_score

(dtree_female_score+dtree_male_score)/2

{'criterion': 'gini',
 'max_depth': 15,
 'min_samples_split': 10,
 'splitter': 'best'}

DecisionTreeClassifier(max_depth=15, min_samples_split=10)

0.7084050101098873

{'criterion': 'gini',
 'max_depth': 15,
 'min_samples_split': 10,
 'splitter': 'best'}

DecisionTreeClassifier(max_depth=15, min_samples_split=10)

0.7084050101098873

0.7084050101098873

In [30]:
# multinomialNB
mnb_params = {
    "alpha": [1.0, 0]
}
mnb= GridSearchCV(MultinomialNB(), mnb_params, scoring = make_scorer(f1_score, average = "macro"), cv = 10, n_jobs = -1)
mnb_female = mnb.fit(df_female_idf.drop(columns = ["gender"]),df_female_idf["gender"])
mnb_male = mnb.fit(df_male_idf.drop(columns = ["gender"]),df_male_idf["gender"])
mnb_female_model = mnb_female.best_estimator_
mnb_male_model = mnb_male.best_estimator_
mnb_female_score = mnb_female.best_score_
mnb_male_score = mnb_male.best_score_
mnb_female_params = mnb_female.best_params_
mnb_male_params = mnb_male.best_params_

mnb_female_params
mnb_female_model
mnb_female_score

mnb_male_params
mnb_male_model
mnb_male_score

(mnb_female_score+mnb_male_score)/2

{'alpha': 0}

MultinomialNB(alpha=0)

0.5143827638341507

{'alpha': 0}

MultinomialNB(alpha=0)

0.5143827638341507

0.5143827638341507

# Implementation:

#### the best result we got was with count vectorizer and multinomialNB model, so that's what we'll use:

In [31]:
#reload data
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [32]:
#clean train set
df_clean=clean_words(df_train)
df_clean

,story,gender
0,אותי אמת חשבתי יקרה אמרתי תיארתי עצני אחרי א...,m
1,פני תגייסתי עשיתי מני יחידות מובחרות סיירות או...,m
2,תחילו חלומו פנגויה קבוצות מכללה אחרי צלחתי ע...,f
3,ייתי אותי טייסת קריאו פני שינה חמשת מלחמות ...,m
4,ייתי מדריכה יינו אני ייתי מדריכה תלמידי יתות א...,f
...,...,...
748,פני טסתי יינו החלטנו אנחנו רצינות דיברנ...,m
749,עליתי ספונטני משפחתי תלבטויות חלטנו אנחנו אילת...,m
750,פני עברנו דירה בית עקבות משפחתי מתגוררת דירה י...,m
751,חוויה אותה ארצה תרחשה פני קיצי קבעתי שבת ...,f


In [33]:
#clean test set
df_test_clean=clean_words(df_test)
df_test_clean

,test_example_id,story
0,0,אני המשפחה ארצות ברית קליפורניה כדי משפחה גר...
1,1,געתי שירות מדינה אחרי תנועת מלאות עשייה אזרחי...
2,2,אחת אהבות גדולות אלו אישתי שנה אחרונה חווי...
3,3,משמעותי עבורי אני קיבלתי תרגשתי הייתי מסג...
4,4,אני געתי מקולומביה אני עשיתי עבודת החלטתי נחתת...
...,...,...
318,318,שנה אחרונה רגשתי עבודה שגרה פכה משהו לתי אית...
319,319,אני מזה חלמנו רקטה אותנו אנחנו רבה צלחנו ישל...
320,320,מידי ישי אני תחבורה ציבורית אני עשות קניות מת...
321,321,פני תחלואה שני עמית עבודה תגלה חולה קורונה יי...


In [34]:
#concat the two sets together
df = pd.concat([df_clean, df_test_clean], axis=0,ignore_index=True)
df

,story,gender,test_example_id
0,אותי אמת חשבתי יקרה אמרתי תיארתי עצני אחרי א...,m,NaN
1,פני תגייסתי עשיתי מני יחידות מובחרות סיירות או...,m,NaN
2,תחילו חלומו פנגויה קבוצות מכללה אחרי צלחתי ע...,f,NaN
3,ייתי אותי טייסת קריאו פני שינה חמשת מלחמות ...,m,NaN
4,ייתי מדריכה יינו אני ייתי מדריכה תלמידי יתות א...,f,NaN
...,...,...,...
1071,שנה אחרונה רגשתי עבודה שגרה פכה משהו לתי אית...,NaN,318.000
1072,אני מזה חלמנו רקטה אותנו אנחנו רבה צלחנו ישל...,NaN,319.000
1073,מידי ישי אני תחבורה ציבורית אני עשות קניות מת...,NaN,320.000
1074,פני תחלואה שני עמית עבודה תגלה חולה קורונה יי...,NaN,321.000


In [35]:
#count vectorize the new dataframe
vec = CountVectorizer(ngram_range=(1,3),min_df=5)
x_train = vec.fit_transform(df["story"])
df_vec = pd.DataFrame(x_train.toarray(), columns=vec.get_feature_names_out())
df_vec["gender"] = df["gender"]
df_vec["test_example_id"] = df["test_example_id"]

In [36]:
#devide the dataframes back to train and test
df_train =  df_vec.dropna(subset=['gender']).copy()
df_train = df_train.drop(columns=["test_example_id"])

In [37]:
df_test =  df_vec.dropna(subset=['test_example_id']).copy()
df_test = df_test.drop(columns=["gender"])

In [38]:
#create the female positive and male positive dataframes
df_female_cvec, df_male_cvec = womanizer(df_train)

In [39]:
#create the best classifier for count vector with best params
clf=MultinomialNB(alpha=0.5)

#### we will continue with the male positive dataframe:

In [40]:
clf.fit(df_male_cvec.drop(columns = ["gender"]), df_male_cvec["gender"])
predicted = clf.predict(df_test.drop(columns = ["test_example_id"]))
predicted

MultinomialNB(alpha=0.5)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,

In [41]:

df_predicted = pd.DataFrame({"test_example_id":df_test["test_example_id"],"predicted_category": predicted})
df_predicted.reset_index(drop=True, inplace=True)
df_predicted['predicted_category'] = df_predicted['predicted_category'].replace({1: 'm', 0: 'f'})

df_predicted

,test_example_id,predicted_category
0,0.000,m
1,1.000,m
2,2.000,m
3,3.000,m
4,4.000,m
...,...,...
318,318.000,m
319,319.000,m
320,320.000,m
321,321.000,m


In [42]:
df_predicted.head()
df_predicted.tail()

,test_example_id,predicted_category
0,0.000,m
1,1.000,m
2,2.000,m
3,3.000,m
4,4.000,m


,test_example_id,predicted_category
318,318.000,m
319,319.000,m
320,320.000,m
321,321.000,m
322,322.000,m


#### for fun we will check the best classifier for TFIDF vector with best params

In [51]:
vec = TfidfVectorizer(ngram_range=(1,3),min_df=5)
x_train = vec.fit_transform(df["story"])
df_vec_idf = pd.DataFrame(x_train.toarray(), columns=vec.get_feature_names_out())
df_vec_idf["gender"] = df["gender"]
df_vec_idf["test_example_id"] = df["test_example_id"]

In [52]:
df_train_idf =  df_vec_idf.dropna(subset=['gender']).copy()
df_train_idf = df_train_idf.drop(columns=["test_example_id"])
df_test_idf =  df_vec_idf.dropna(subset=['test_example_id']).copy()
df_test_idf = df_test_idf.drop(columns=["gender"])

In [53]:
df_female_idf, df_male_idf= womanizer(df_train_idf)

In [54]:
clf_tfidf=DecisionTreeClassifier(criterion= 'gini',max_depth=15,min_samples_split= 2,splitter= 'best')

In [55]:
clf_tfidf.fit(df_male_idf.drop(columns = ["gender"]), df_male_idf["gender"])
predicted1 = clf_tfidf.predict(df_test_idf.drop(columns = ["test_example_id"]))
predicted1

DecisionTreeClassifier(max_depth=15)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,

In [56]:
df_predicted2 = pd.DataFrame({"test_example_id":df_test_idf["test_example_id"],"predicted_category": predicted1})
df_predicted2.reset_index(drop=True, inplace=True)
df_predicted2['predicted_category'] = df_predicted2['predicted_category'].replace({1: 'm', 0: 'f'})

df_predicted2

,test_example_id,predicted_category
0,0.000,m
1,1.000,m
2,2.000,m
3,3.000,m
4,4.000,m
...,...,...
318,318.000,m
319,319.000,m
320,320.000,f
321,321.000,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [46]:
df_predicted.to_csv('classification_results.csv',index=False)